In [0]:
%run  /brewdat-global-sales-bees/framework-bees/library_v1.0.0/main

In [0]:
dbutils.widgets.text("watermark_start_datetime", "2022-01-14 00:00:00")
watermark_start_datetime = dbutils.widgets.get("watermark_start_datetime")
print(f"watermark_start_datetime: {watermark_start_datetime}")

dbutils.widgets.text("watermark_end_datetime", "2022-01-14 23:59:59")
watermark_end_datetime = dbutils.widgets.get("watermark_end_datetime")
print(f"watermark_end_datetime: {watermark_end_datetime}")

In [0]:
try:
    from pyspark.sql import functions as F

    key_columns = ["customer_bk"]
    partition_columns = ["extract_yr","extract_mth","extract_day"]

    df = spark.sql(f"""
        SELECT
            CAST(UNIX_TIMESTAMP(dt_reference_ts, 'yyyyMMddHHmmss') AS TIMESTAMP) AS dt_reference_ts,
            srs_country_cd,
            src_region_nm,
            src_sys_nm,
            channel_cd,
            channel_ds,
            channel_typ,
            city_nm,
            address_ds,
            CAST(coordinate_lat AS DECIMAL(15,2)) AS coordinate_lat,
            CAST(coordinate_log AS DECIMAL(15,2)) AS coordinate_log,
            country_cd,
            CAST(credit_limit_val AS DECIMAL(15,2)) AS credit_limit_val,
            credit_payment_terms_typ,
            customer_bees_cd,
            customer_bk,
            customer_erp_cd,
            customer_nm,
            customer_socioeconomic_cls,
            customer_potential_cls,
            customer_stt,
            customer_updated_dtm,
            segment_cd,
            payment_methods_cls,
            poc_size_cls,
            segment_nm,
            state_nm,
            subchannel_cd,
            subchannel_nm,
            zipcode_cd,
            extract_version_cd,
            extract_day,
            extract_yr,
            extract_mth
        FROM
            abi_sales_maz_co_lz.co_customer_lz
        WHERE 1 = 1
            AND extract_yr BETWEEN DATE_FORMAT('{watermark_start_datetime}', 'yyyy')
                AND DATE_FORMAT('{watermark_end_datetime}', 'yyyy')
            AND extract_mth BETWEEN DATE_FORMAT('{watermark_start_datetime}', 'yyyyMM')
                AND DATE_FORMAT('{watermark_end_datetime}', 'yyyyMM')
            AND extract_day BETWEEN DATE_FORMAT('{watermark_start_datetime}', 'yyyyMMdd')
                AND DATE_FORMAT('{watermark_end_datetime}', 'yyyyMMdd')
    """)

except:
    Framework.exit_with_last_exception()

#display(df)testehistorico

In [0]:
df = Framework.deduplicate_records(
    df=df,
    key_columns=key_columns,
    watermark_column="dt_reference_ts",
)

#display(df)

In [0]:
df = Framework.create_or_replace_audit_columns(df)

#display(df)

In [0]:
schema_name = "abi_sales_maz_co_hz"
table_name = "co_customer_hz"

location = Framework.generate_silver_table_location(
    source_zone="maz-co",
    source_sys_name="file_bees",
    source_business_domain="sales",
    table_name=table_name,
)

results = Framework.write_delta_table(
    df=df,
    location=location,
    schema_name=schema_name,
    table_name=table_name,
    load_type=Framework.LoadType.UPSERT,
    key_columns=key_columns,
    partition_columns=partition_columns,
    schema_evolution_mode=Framework.SchemaEvolutionMode.ADD_NEW_COLUMNS,
)

print(results)

In [0]:
Framework.exit_with_object(results)
